In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:50000]
y_train = y[:50000]
X_test = X[50000:60000]
y_test = y[50000:60000]
X_valid = X[60000:]
y_valid = y[60000:]

In [4]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.metrics import accuracy_score

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
etc = ExtraTreesClassifier(n_estimators=100, random_state=42)

voting = VotingClassifier(
    estimators=[('rfc', rfc), ('etc', etc)],
    voting='hard'
)

for clf in (rfc, etc, voting):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

RandomForestClassifier 0.968
ExtraTreesClassifier 0.9703
VotingClassifier 0.9691


In [5]:
y_pred = etc.predict(X_test)
accuracy_score(y_test, y_pred)

0.9743

In [6]:
estimators = [rfc, etc]

X_val_predictions = np.empty((len(X_valid), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_valid)

In [7]:
X_val_predictions

array([[7., 7.],
       [2., 2.],
       [1., 1.],
       ...,
       [4., 4.],
       [5., 5.],
       [6., 6.]], dtype=float32)

In [8]:
etc = ExtraTreesClassifier(n_estimators=100, random_state=42, oob_score=True, bootstrap=True)
etc.fit(X_val_predictions, y_valid)
etc.oob_score_

0.9697

In [9]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [10]:
y_pred = etc.predict(X_test_predictions)

In [11]:
accuracy_score(y_test, y_pred)

0.9741